In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
from plots_file import *
from our_functionsv3 import *

np.random.seed(0)
import tensorflow.keras as keras
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
from keras.callbacks import EarlyStopping
import time
import matplotlib
import matplotlib.pyplot as plt

C:\Users\Pauli\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from LSTM_functions import *

### Load data

In [4]:
X_train, Y_train = get_data(pos = "twitter-datasets/train_pos_preprocessed.txt", neg = "twitter-datasets/train_neg_preprocessed.txt")
Y_train_oh = convert_to_one_hot(Y_train, C=2)

In [5]:
ids, _ = get_test_data("twitter-datasets/test_data.txt")
X_test = read_data("twitter-datasets/test_preprocessed.txt")

In [6]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('glove.6B.50d.txt')

### Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences

In [7]:
max_length = get_max_length(X_train)

In [8]:
X_train_indices = sentences_to_indices(X_train, word_to_index, max_length)
X_test_indices = sentences_to_indices(X_test, word_to_index, max_length)

### Submission

In [9]:
histories = []
models = []

optimizer = ['RMSprop', 'Adagrad', 'Adadelta', 'Adamax', 'Nadam']
for opt in optimizer:
    history, model = complete_model(X_train_indices, Y_train_oh, word_to_vec_map, word_to_index, max_length, summary = False, dropout_rate = 0.5, batch_size = 128, 
                                     epochs = 50, loss ='binary_crossentropy', optimizer = opt)
    histories.append(history)
    models.append(model)
    label = new_predict_lstm(model, X_test_indices)
    path = 'submissions/submission_model_optimizer_'+ opt + '_.csv'
    create_csv_submission(ids, label, path)

Train on 180000 samples, validate on 20000 samples
Epoch 1/50
180000/180000 [==============================] - 797s 4ms/step - loss: 0.5210 - acc: 0.7248 - val_loss: 0.4849 - val_acc: 0.7575
Epoch 2/50
180000/180000 [==============================] - 777s 4ms/step - loss: 0.4768 - acc: 0.7606 - val_loss: 0.4718 - val_acc: 0.7651
Epoch 3/50
180000/180000 [==============================] - 781s 4ms/step - loss: 0.4594 - acc: 0.7744 - val_loss: 0.4577 - val_acc: 0.7772
Epoch 4/50
180000/180000 [==============================] - 775s 4ms/step - loss: 0.4454 - acc: 0.7847 - val_loss: 0.4757 - val_acc: 0.7673
Epoch 5/50
180000/180000 [==============================] - 929s 5ms/step - loss: 0.4352 - acc: 0.7913 - val_loss: 0.4453 - val_acc: 0.7883
Epoch 6/50
180000/180000 [==============================] - 1020s 6ms/step - loss: 0.4299 - acc: 0.7967 - val_loss: 0.4425 - val_acc: 0.7851
Epoch 7/50
180000/180000 [==============================] - 1080s 6ms/step - loss: 0.4234 - acc: 0.8009 - va

180000/180000 [==============================] - 719s 4ms/step - loss: 0.6657 - acc: 0.5532 - val_loss: 0.5645 - val_acc: 0.6923
Epoch 2/50
180000/180000 [==============================] - 719s 4ms/step - loss: 0.5008 - acc: 0.7418 - val_loss: 0.4704 - val_acc: 0.7615
Epoch 3/50
180000/180000 [==============================] - 716s 4ms/step - loss: 0.4597 - acc: 0.7716 - val_loss: 0.4575 - val_acc: 0.7689
Epoch 4/50
180000/180000 [==============================] - 714s 4ms/step - loss: 0.5004 - acc: 0.7394 - val_loss: 0.4929 - val_acc: 0.7469
Epoch 5/50
180000/180000 [==============================] - 717s 4ms/step - loss: 0.4562 - acc: 0.7764 - val_loss: 0.4562 - val_acc: 0.7781
Epoch 6/50
180000/180000 [==============================] - 715s 4ms/step - loss: 0.4299 - acc: 0.7931 - val_loss: 0.4404 - val_acc: 0.7860
Epoch 7/50
180000/180000 [==============================] - 716s 4ms/step - loss: 0.4161 - acc: 0.8020 - val_loss: 0.4333 - val_acc: 0.7864
Epoch 8/50
180000/180000 [=====

In [27]:
histories = []
models = []

optimizer = ['adam']
for opt in optimizer:
    history, model = complete_model(X_train_indices, Y_train_oh, word_to_vec_map, word_to_index, max_length, summary = False, dropout_rate = 0.5, batch_size = 128, 
                                     epochs = 50, loss ='binary_crossentropy', optimizer = opt)
    histories.append(history)
    models.append(model)
    label = new_predict_lstm(model, X_test_indices)
    path = 'submissions/submission_model_1_output_.csv'
    create_csv_submission(ids, label, path)